In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold, cross_val_predict, train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

import math

from copy import deepcopy

In [2]:
spectra = pd.read_excel('pi_cup.xlsx')

spectra.drop(columns=['Unnamed: 0'], inplace=True)

spectra

,Ch_0_0,Ch_0_1,Ch_0_2,Ch_0_3,Ch_0_4,Ch_0_5,Ch_0_6,Ch_0_7,Ch_0_8,Ch_0_9,...,Hyaline cylinders_a,Bacteria_a,Crystals_a,Ferment_a,Small cells_a,Pathological cylinders_a,Slime_a,Spermatozoon_a,TOTAL_a,TOTAL
0,116.2066,86.67870,501.8073,1966.517,1211.834,3746.119,420.2404,1028.663,1396.969,233.9503,...,1,1,0,0,0,1,0,0,8,1
1,110.4915,89.53624,495.1797,2054.570,1219.735,3993.029,410.4901,1043.288,1414.642,244.8904,...,1,1,0,0,0,1,0,0,8,1
2,103.8239,81.91614,484.7648,1963.676,1213.810,3925.869,400.7397,1040.363,1449.987,248.2566,...,1,1,0,0,0,1,0,0,8,1
3,109.5390,85.72619,510.3286,2041.314,1270.105,4032.534,412.4402,1055.964,1477.758,249.0982,...,1,1,0,0,0,1,0,0,8,1
4,105.7290,82.86865,488.5520,1967.463,1214.797,3926.857,408.5400,1042.313,1459.244,248.2566,...,1,1,0,0,0,1,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,131.4468,110.49150,632.4666,2789.291,1586.150,5790.534,458.2668,1126.166,1433.998,253.3059,...,0,1,0,0,1,0,0,0,5,1
4228,131.4468,110.49150,633.4134,2793.078,1586.150,5820.163,459.2419,1123.241,1433.998,251.6228,...,0,1,0,0,1,0,0,0,5,1
4229,131.4468,109.53900,634.3602,2764.674,1585.162,5782.632,459.2419,1124.216,1432.315,252.4644,...,0,1,0,0,1,0,0,0,5,1
4230,132.3993,109.53900,636.2538,2770.355,1588.125,5762.879,460.2169,1123.241,1437.364,252.4644,...,0,1,0,0,1,0,0,0,5,1


In [3]:
X=spectra.iloc[:,0:36]
X

,Ch_0_0,Ch_0_1,Ch_0_2,Ch_0_3,Ch_0_4,Ch_0_5,Ch_0_6,Ch_0_7,Ch_0_8,Ch_0_9,...,Ch_1_8,Ch_1_9,Ch_1_10,Ch_1_11,Ch_1_12,Ch_1_13,Ch_1_14,Ch_1_15,Ch_1_16,Ch_1_17
0,116.2066,86.67870,501.8073,1966.517,1211.834,3746.119,420.2404,1028.663,1396.969,233.9503,...,241.2221,660.4495,738.9610,748.5693,419.7867,606.7318,113.9607,206.4316,103.15650,146.9493
1,110.4915,89.53624,495.1797,2054.570,1219.735,3993.029,410.4901,1043.288,1414.642,244.8904,...,232.0723,638.8227,729.3528,700.5281,408.5924,614.5677,118.5191,212.2925,106.07600,155.7079
2,103.8239,81.91614,484.7648,1963.676,1213.810,3925.869,400.7397,1040.363,1449.987,248.2566,...,232.0723,661.2813,748.5693,736.3406,400.7564,640.3147,118.5191,227.2702,107.04920,140.1371
3,109.5390,85.72619,510.3286,2041.314,1270.105,4032.534,412.4402,1055.964,1477.758,249.0982,...,231.2405,657.1223,756.4305,726.7324,415.3090,632.4786,121.1239,224.6654,108.02230,163.4933
4,105.7290,82.86865,488.5520,1967.463,1214.797,3926.857,408.5400,1042.313,1459.244,248.2566,...,236.2313,671.2629,765.1653,750.3162,408.5924,574.2682,119.1703,233.7822,107.04920,142.0834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,131.4468,110.49150,632.4666,2789.291,1586.150,5790.534,458.2668,1126.166,1433.998,253.3059,...,231.2405,586.4193,696.1607,647.2460,413.0701,589.9402,114.6119,197.9660,99.26377,140.1371
4228,131.4468,110.49150,633.4134,2793.078,1586.150,5820.163,459.2419,1123.241,1433.998,251.6228,...,230.4087,578.9330,693.5402,660.3481,411.9507,548.5213,114.6119,216.1997,99.26377,130.4053
4229,131.4468,109.53900,634.3602,2764.674,1585.162,5782.632,459.2419,1124.216,1432.315,252.4644,...,230.4087,591.4100,694.4138,654.2338,411.9507,608.9706,113.9607,203.1756,99.26377,124.5663
4230,132.3993,109.53900,636.2538,2770.355,1588.125,5762.879,460.2169,1123.241,1437.364,252.4644,...,231.2405,592.2418,693.5402,683.9320,413.0701,569.7905,113.9607,193.4076,99.26377,136.2444


In [5]:
y1 = spectra.iloc[:,[39,40]]
y2 = spectra.iloc[:,[41,42,43,44,46,47]]
y3 = spectra.iloc[:,[45,48,49,50,51,52,53,54,55,56,57,58,59]]
y3

,Leukocyte_a,Blood_a,Erythrocyte_a,Leukocyte_a.1,Squamous cells_a,Hyaline cylinders_a,Bacteria_a,Crystals_a,Ferment_a,Small cells_a,Pathological cylinders_a,Slime_a,Spermatozoon_a
0,1,1,1,1,0,1,1,0,0,0,1,0,0
1,1,1,1,1,0,1,1,0,0,0,1,0,0
2,1,1,1,1,0,1,1,0,0,0,1,0,0
3,1,1,1,1,0,1,1,0,0,0,1,0,0
4,1,1,1,1,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,0,0,1,1,1,0,1,0,0,1,0,0,0
4228,0,0,1,1,1,0,1,0,0,1,0,0,0
4229,0,0,1,1,1,0,1,0,0,1,0,0,0
4230,0,0,1,1,1,0,1,0,0,1,0,0,0


In [6]:
def sigmoid(x):
    return 1 /(1+(math.e**-x))

In [7]:
def medical_metrics(true, pred):
    sensitivity =  (pred * true).sum() / true.sum()
    specificity = ((1 - pred)*(1-true)).sum() / (1 - true).sum()
    return sensitivity, specificity

In [8]:
def print_metrics(true, pred):
    for i in range(len(true.columns)):
        y_true = true.iloc[:,i]
        y_pred = pred.iloc[:,i]
        anomaly = list(true)[i]
        sens, spec = medical_metrics(y_true, y_pred)
        print('\nАномалия:\t', anomaly)
        print('Чувствительность:\t', sens)
        print('Специфичность:\t', spec)

In [9]:
def pls_func(X_train, X_test, y_train, y_test, n_comp):
    pls = PLSRegression(n_components=n_comp)
    pls.fit(X_train,y_train)
    pls_result = pls.predict(X_test)
    pls_sigmoid = sigmoid(pls_result)
    binary_prediction = (pls_sigmoid > 0.5).astype('uint8')
    y_pred = pd.DataFrame(binary_prediction)

    return y_pred

In [10]:
def get_components_data(X_train, X_test, y_train, y_test, n_min, n_max):
    results_list = []
    results_list.append(['n_comp','Anomaly','sens','spec'])
    for i in range(n_max-n_min+1):
        y_pred = pls_func(X_train, X_test, y_train, y_test, i+n_min)
        for j in range(len(y_test.columns)):
            true = y_test.iloc[:,j]
            pred = y_pred.iloc[:,j]
            anomaly = list(y_test)[j]
            sens, spec = medical_metrics(true, pred)
            results_list.append([i+n_min,anomaly,sens,spec])
    components_data = pd.DataFrame(results_list)
    new_header = components_data.iloc[0]
    components_data = components_data[1:]
    components_data.columns = new_header
    return components_data


In [11]:
def components_selection(components_data):
    anomaly_list = components_data.drop_duplicates('Anomaly').Anomaly
    for i in range(len(anomaly_list)):
        max_sens = components_data.loc[components_data['Anomaly'] == anomaly_list[i+1]].sort_values('sens').sens.max()
        components_list = components_data.loc[components_data['Anomaly'] == anomaly_list[i+1]].loc[components_data['sens'] == max_sens].n_comp
        print('\nПараметр аномалии\t', anomaly_list[i+1])
        print('MAX чувствительность\t', max_sens)
        print('Число компонент\n')
        print(components_list)

In [12]:
# Test-train split
X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(X, y1, y2, y3, test_size=0.2,)

In [13]:
components_data1 = get_components_data(X_train, X_test, y1_train, y1_test,2,36)
components_data2 = get_components_data(X_train, X_test, y2_train, y2_test,2,36)
components_data3 = get_components_data(X_train, X_test, y3_train, y3_test,2,36)

/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: inva

In [14]:
# Выбор компонент для 1 категории признаков
components_selection(components_data1)


Параметр аномалии	 Density_a
MAX чувствительность	 0.21839080459770116
Число компонент

1      2
3      3
5      4
7      5
9      6
11     7
13     8
15     9
17    10
19    11
21    12
23    13
25    14
27    15
29    16
31    17
33    18
35    19
37    20
49    26
51    27
53    28
57    30
Name: n_comp, dtype: object

Параметр аномалии	 pH_a
MAX чувствительность	 0.09090909090909091
Число компонент

2      2
4      3
6      4
8      5
10     6
12     7
14     8
16     9
18    10
20    11
22    12
24    13
26    14
28    15
30    16
32    17
34    18
36    19
38    20
40    21
42    22
44    23
46    24
48    25
50    26
52    27
54    28
56    29
58    30
60    31
62    32
64    33
66    34
68    35
70    36
Name: n_comp, dtype: object


In [15]:
# Выбор компонент для 2 категории признаков
components_selection(components_data2)


Параметр аномалии	 Protein_a
MAX чувствительность	 0.3076923076923077
Число компонент

13     4
19     5
31     7
79    15
Name: n_comp, dtype: object

Параметр аномалии	 Bilirubin_a
MAX чувствительность	 nan
Число компонент

Series([], Name: n_comp, dtype: object)

Параметр аномалии	 Glucose_a
MAX чувствительность	 0.5185185185185185
Число компонент

3    2
9    3
Name: n_comp, dtype: object

Параметр аномалии	 Ketones_a
MAX чувствительность	 0.5217391304347826
Число компонент

10     3
22     5
34     7
64    12
70    13
Name: n_comp, dtype: object

Параметр аномалии	 Nitrite_a
MAX чувствительность	 0.55
Число компонент

23      5
29      6
59     11
77     14
83     15
89     16
119    21
125    22
131    23
137    24
143    25
149    26
161    28
167    29
173    30
Name: n_comp, dtype: object

Параметр аномалии	 Urobilinogen_a
MAX чувствительность	 0.29411764705882354
Число компонент

6     2
12    3
Name: n_comp, dtype: object


In [15]:
# Выбор компонент для 3 категории признаков
components_selection(components_data3)


Параметр аномалии	 Leukocyte_a
MAX чувствительность	 0.4444444444444444
Число компонент

1     2
14    3
27    4
40    5
Name: n_comp, dtype: object

Параметр аномалии	 Blood_a
MAX чувствительность	 0.37623762376237624
Число компонент

2     2
15    3
28    4
Name: n_comp, dtype: object

Параметр аномалии	 Erythrocyte_a
MAX чувствительность	 0.30434782608695654
Число компонент

3    2
Name: n_comp, dtype: object

Параметр аномалии	 Leukocyte_a.1
MAX чувствительность	 0.33766233766233766
Число компонент

4     2
17    3
30    4
43    5
Name: n_comp, dtype: object

Параметр аномалии	 Squamous cells_a
MAX чувствительность	 0.34285714285714286
Число компонент

5       2
96      9
109    10
122    11
Name: n_comp, dtype: object

Параметр аномалии	 Hyaline cylinders_a
MAX чувствительность	 0.46153846153846156
Число компонент

6       2
19      3
110    10
123    11
136    12
Name: n_comp, dtype: object

Параметр аномалии	 Bacteria_a
MAX чувствительность	 0.22900763358778625
Число компонент


In [16]:
model_data1 = get_components_data(X_train, X_test, y1_train, y1_test,2,8)
model_data2 = get_components_data(X_train, X_test, y2_train, y2_test,2,8)
model_data3 = get_components_data(X_train, X_test, y3_train, y3_test,1,8)

/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: inva

In [17]:
#Данные по 1 категории признаков
model_data1.sort_values('Anomaly')

,n_comp,Anomaly,sens,spec
1,2,Density_a,0.218391,0
3,3,Density_a,0.218391,0
5,4,Density_a,0.218391,0.00200401
7,5,Density_a,0.218391,0.00200401
9,6,Density_a,0.218391,0.00200401
11,7,Density_a,0.218391,0.00200401
13,8,Density_a,0.218391,0.00200401
2,2,pH_a,0.0909091,0.0351515
4,3,pH_a,0.0909091,0.0387879
6,4,pH_a,0.0909091,0.0472727


In [18]:
#Данные по 2 категории признаков
model_data2.sort_values('Anomaly')

,n_comp,Anomaly,sens,spec
2,2,Bilirubin_a,NaN,0.197166
26,6,Bilirubin_a,NaN,0.197166
32,7,Bilirubin_a,NaN,0.197166
20,5,Bilirubin_a,NaN,0.197166
8,3,Bilirubin_a,NaN,0.197166
14,4,Bilirubin_a,NaN,0.197166
38,8,Bilirubin_a,NaN,0.197166
21,5,Glucose_a,0.347826,0.026699
27,6,Glucose_a,0.304348,0.0339806
33,7,Glucose_a,0.304348,0.0339806


In [19]:
#Данные по 3 категории признаков
model_data3.sort_values('Anomaly')

,n_comp,Anomaly,sens,spec
85,7,Bacteria_a,0.21374,0.023743
72,6,Bacteria_a,0.206107,0.0195531
46,4,Bacteria_a,0.206107,0.0209497
98,8,Bacteria_a,0.206107,0.0251397
59,5,Bacteria_a,0.206107,0.0195531
...,...,...,...,...
96,8,Squamous cells_a,0.314286,0.023399
31,3,Squamous cells_a,0.314286,0.0172414
18,2,Squamous cells_a,0.342857,0.00369458
5,1,Squamous cells_a,0.257143,0.0135468
